In [115]:
import csv
import operator
import matplotlib.pyplot as plt
import pandas as pd
import operator
import scipy.stats as stats
import math
import numpy as np
import random
import stv
import itertools

In [116]:
#read files
rcv = pd.read_csv('rcv_elections.csv')
E03_cam = pd.read_csv('Cambridge_City_Council_CVR_2003.csv')
E05_cam= pd.read_csv('Cambridge_City_Council_CVR_2005.csv')
E07_cam= pd.read_csv('Cambridge_City_Council_CVR_2007.csv')
E09_cam= pd.read_csv('Cambridge_City_Council_CVR_2009.csv')
E11_cam= pd.read_csv('Cambridge_City_Council_CVR_2011.csv')
E13_cam= pd.read_csv('Cambridge_City_Council_CVR_2013.csv')
E15_cam= pd.read_csv('Cambridge_City_Council_CVR_2015.csv')
E17_cam= pd.read_csv('Cambridge_City_Council_CVR_2017.csv')

In [117]:
#data cleaning 
E03_cam['Precinct'] = E03_cam['ID'].map(lambda x: str(x)[:-8]).str.lstrip('0')
E05_cam['Precinct'] = E05_cam['ID'].map(lambda x: str(x)[:-8]).str.lstrip('0')
E07_cam['Precinct'] = E07_cam['ID'].map(lambda x: str(x)[:-8]).str.lstrip('0')
E09_cam['Precinct'] = E09_cam['ID'].map(lambda x: str(x)[:-21]).str.lstrip('0')
E11_cam['Precinct'] = E11_cam['ID'].map(lambda x: str(x)[:-8]).str.lstrip('0')
E13_cam['Precinct'] = E13_cam['ID'].map(lambda x: str(x)[:-21]).str.lstrip('0')

In [118]:
#E03_cam.to_csv('cleaned_E03_cam.csv')
#E05_cam.to_csv('cleaned_E05_cam.csv')
#E07_cam.to_csv('cleaned_E07_cam.csv')
#E09_cam.to_csv('cleaned_E09_cam.csv')
#E11_cam.to_csv('cleaned_E11_cam.csv')
#E13_cam.to_csv('cleaned_E13_cam.csv')

### Experiment Set Up

In [119]:
def set_up(election_file_name, precinct_column, preference_columns, num_cands_for_cutoff):
    precincts = set()
    precinct_votes = {}
    precinct_votes_tally = {}
    cands = set()
    precinct_cands_tally = {}

    with open(election_file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                if row[precinct_column] not in precincts:
                    preferences = [row[x] for x in preference_columns]
                    precincts.add(row[precinct_column])
                    precinct_votes[row[precinct_column]] = [preferences]
                    precinct_votes_tally[row[precinct_column]] = {}
                    precinct_cands_tally[row[precinct_column]] = {}
                if tuple(preferences) not in precinct_votes_tally[row[precinct_column]]:
                    precinct_votes_tally[row[precinct_column]][tuple(preferences)] = 1
    
                else:
                    precinct_votes[row[precinct_column]].append(preferences)
                    cands.update([row[x] for x in preference_columns])
                    precinct_votes_tally[row[precinct_column]][tuple(preferences)] += 1
                for i in range(len(preference_columns)):
                    if row[preference_columns[i]] not in precinct_cands_tally[row[precinct_column]]:
                        precinct_cands_tally[row[precinct_column]][row[preference_columns[i]]] = [0]*len(preference_columns)
                    precinct_cands_tally[row[precinct_column]][row[preference_columns[i]]][i] += 1
                
                line_count += 1
    
    prec_STV = {}

    for prec in precincts:
        prec_ballot_list = []
        for item in precinct_votes_tally[prec].items():
            prec_ballot_list.append([list(item[0]), item[1]])
        ranking = stv.rcv_run(prec_ballot_list, num_cands_for_cutoff, True, False)
        winners = ranking[0]
        losers = ranking[1]	
        losers.reverse()
        prec_STV[prec] = winners+losers
    
    
    return precincts, precinct_votes, cands,precinct_cands_tally, prec_STV

def clean_cands(cand_partial):
    for i in cands:
        if i not in cand_partial:
            cand_partial.append(i)
    if '0' in cand_partial:
        cand_partial.remove('0') #takes out nothing candidate
    return cand_partial  




In [120]:
def borda_score(point_scheme, precinct_votes, prec_STV_dic, dist_func, obj_func):
    """
    scores the difference between Borda and STV for each precinct for a given borda point distribution and election. 
    
    point_scheme:  list of associated point values for 1st place thru kth place, given k candidates (p_1, ..., p_k)
    precinct_votes: dictionary with precincts as keys and a dictionary of how many 1st, 2nd,...,nth place votes 
                    they recieved
    prec_STV_dic: dictionary with precincts as keys and a list of individual ballots for each vote in that precinct,
                under an STV system
    dist_func: function that takes the Borda point and STV preference schedules and quantifies the dist between them
    obj_func: funcation that aggregates the distance results from each precinct 
    
    """
        
    borda_dict = {}
    borda_result_dict = {}
    precinct_dists = []
    for precinct in list(precinct_votes.keys()): #iter thru each precinct
        borda_dict[precinct] = {}
        #print(precinct_votes[precinct])
        for cand in list(precinct_votes[precinct]):
            borda_dict[precinct][cand] = np.dot(precinct_votes[precinct][cand], point_scheme)
        
        #sort total points for each precinct to find out the outcome
        sorted_borda_dict = dict(sorted(borda_dict[precinct].items(), key=operator.itemgetter(1), reverse=True)) 
        
        
        borda_result_dict[precinct] = list(sorted_borda_dict.keys())
    
        precinct_dists.append(dist_func(clean_cands(borda_result_dict[precinct]), clean_cands(prec_STV_dic[precinct])))
    
    score = obj_func(precinct_dists)
    
    return score

In [121]:
### distance functions

def kendall_tau(x1, x2):
    tau, p = stats.kendalltau(x1, x2)
    return (-tau+1)/2 #move the tau distribution to be [0-1]

In [122]:
### objective functions

def obj_1():
    return 1

def L1_norm(val_list):
    return sum(abs(i) for i in val_list)


def L2_norm(val_list):
    running_sum = 0
    for i in val_list:
        running_sum += abs(i)**2
    return running_sum ** (1/2)

In [114]:
###wrapper functions from https://github.com/amybecker/RCV/blob/master/borda_experiment.py

# from https://stackoverflow.com/questions/28965734/general-bars-and-stars/28969798#28969798
def partitions(n, k):
    for c in itertools.combinations(range(n+k-1), k-1):
        yield [b-a-1 for a, b in zip((-1,)+c, c+(n+k-1,))]

def borda_optimization_wrapper_3cand(num_cands, points_total, election_data, prec_STV_dic, dist_func, obj_func):
    '''
    Returns borda point distribution that best estimates the STV outcome for a specific RCV election
    ****hard coded for case when electing 3 candidates****
    
    num_cands: number of candidates running, int
    points_total: total sum of all borda points
    precinct_votes: dictionary with precincts as keys and a dictionary of how many 1st, 2nd,...,nth place votes 
                    they recieved
    prec_STV_dic: dictionary with precincts as keys and a list of individual ballots for each vote in that precinct,
                under an STV system
    dist_func: function that takes the Borda point and STV preference schedules and quantifies the dist between them
    obj_func: funcation that aggregates the distance results from each precinct

    '''
    
    best_val = math.inf
    best_option = [-1]*num_cands
    for i in range(points_total+1):
        for j in range(points_total-i+1):
            k = points_total-i-j
            point_scheme = [i,j,k] + [0]*(num_cands-3)
            cur_val = borda_score(point_scheme, precinct_votes, prec_STV_dic, dist_func, obj_func)
            print(point_scheme, cur_val)
            if cur_val < best_val:
                best_option = point_scheme
                best_val = cur_val
    return best_options

def borda_optimization_wrapper_ncand(n, num_cands, points_total, election_data, prec_STV_dic, dist_func, obj_func):
    ''' 
    Returns the borda point distribution for electing n candidates that best estimates the actual STV outcome for a specific RCV election
    
    n: number of candidates to be elected
    num_cands: number of candidates running, int
    points_total: total sum of all borda points
    precinct_votes: dictionary with precincts as keys and a dictionary of how many 1st, 2nd,...,nth place votes 
                    they recieved
    prec_STV_dic: dictionary with precincts as keys and a list of individual ballots for each vote in that precinct,
                under an STV system
    dist_func: function that takes the Borda point and STV preference schedules and quantifies the dist between them
    obj_func: funcation that aggregates the distance results from each precinct
    '''
    
    best_val = math.inf
    best_option = [-1]*num_cands
    #with open(output_file_name, 'w') as writeFile:
        #writer = csv.writer(writeFile)
    for vec in partitions(points_total, n):
        #print(vec[0])
        point_scheme = vec + [0]*(num_cands-n)
        cur_val = borda_score(point_scheme, election_data, prec_STV_dic, dist_func, obj_func)
        #writer.writerow(point_scheme + [cur_val])
        if cur_val < best_val:
            best_option = point_scheme
            best_val = cur_val

    return best_option

### Experiment on Cambridge Data

In [95]:
### 2003 City Council Election Cambridge

election_file_name = 'cleaned_E03_cam.csv'
precinct_column = 23
preference_columns = np.arange(3, 23)
precincts, precinct_votes, cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)



In [97]:
points_to_distribute = 10
num_ranked = len(preference_columns)


E03_optimized =  borda_optimization_wrapper_ncand(6, 
                                                num_ranked, 
                                                points_to_distribute,
                                                precinct_cands_tally, 
                                                prec_STV,  
                                                kendall_tau, 
                                                L1_norm, 
                                                )

In [98]:
E03_optimized

[4, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [99]:
### 2005 City Council Election Cambridge

election_file_name = 'cleaned_E05_cam.csv'
precinct_column = 21
preference_columns = np.arange(3, 21)
precincts, precinct_votes,cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)

points_to_distribute = 10
num_ranked = len(preference_columns)


E05_optimized =  borda_optimization_wrapper_ncand(6, 
                                                 len(preference_columns), 
                                                 points_to_distribute,
                                                 precinct_cands_tally, 
                                                 prec_STV,  
                                                 kendall_tau, 
                                                 L1_norm, 
                                                )

In [100]:
### 2007 City Council Election Cambridge

election_file_name = 'cleaned_E07_cam.csv'
precinct_column = 18
preference_columns = np.arange(3, 18)
precincts, precinct_votes, cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)

points_to_distribute = 10
num_ranked = len(preference_columns)

E07_optimized =  borda_optimization_wrapper_ncand(6, 
                                                 num_ranked, 
                                                 points_to_distribute,
                                                 precinct_cands_tally, 
                                                 prec_STV,  
                                                 kendall_tau, 
                                                 L1_norm, 
                                                )

In [101]:
### 2009 City Council Election Cambridge

election_file_name = 'cleaned_E09_cam.csv'
precinct_column = 23
preference_columns = np.arange(3, 23)
precincts, precinct_votes, cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)

points_to_distribute = 10
num_ranked = len(preference_columns)

E09_optimized = borda_optimization_wrapper_ncand(6, 
                                                 num_ranked, 
                                                 points_to_distribute,
                                                 precinct_cands_tally, 
                                                 prec_STV,  
                                                 kendall_tau, 
                                                 L1_norm, 
                                                )

In [102]:
### 2011 City Council Election Cambridge

election_file_name = 'cleaned_E11_cam.csv'
precinct_column = 21
preference_columns = np.arange(3, 21)
precincts, precinct_votes,cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)


points_to_distribute = 10
num_ranked = len(preference_columns)

E11_optimized = borda_optimization_wrapper_ncand(6, 
                                                 num_ranked, 
                                                 points_to_distribute,
                                                 precinct_cands_tally, 
                                                 prec_STV,  
                                                 kendall_tau, 
                                                 L1_norm, 
                                                )

In [103]:
### 2013 City Council Election Cambridge

election_file_name = 'cleaned_E13_cam.csv'
precinct_column = 27
preference_columns = np.arange(3, 27)
precincts, precinct_votes, cands,precinct_cands_tally,  prec_STV = set_up(election_file_name,
                                                                                               precinct_column,
                                                                                               preference_columns,
                                                                                               9)

points_to_distribute = 10
num_ranked = len(preference_columns)

E13_optimized = borda_optimization_wrapper_ncand(6, 
                                                 num_ranked, 
                                                 points_to_distribute,
                                                 precinct_cands_tally, 
                                                 prec_STV,  
                                                 kendall_tau, 
                                                 L1_norm, 
                                                )

In [104]:
print('best 2003:', 
      E03_optimized)
print('best 2005:',
     E05_optimized)
print('best 2007:',
     E07_optimized)
print('best 2009:',
     E09_optimized)
print('best 2011:',
     E11_optimized)
print('best 2013:',
     E13_optimized)

best 2003: [4, 1, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
best 2005: [2, 7, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
best 2007: [0, 3, 0, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
best 2009: [0, 3, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
best 2011: [10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
best 2013: [1, 2, 5, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
